In [ ]:
import os
import pickle as pk
from ipywidgets import interact
from copy import deepcopy

from tqdm import tqdm_notebook as tqdm

import xarray as xr
import numpy as np
import pandas as pd

import distributed

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.colors
import cartopy.crs as ccrs
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters

In [ ]:
rnd = np.random.RandomState(42)

In [ ]:
DENSITY = 1000

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')
register_matplotlib_converters()
plt.rcParams['text.usetex'] = False

In [ ]:
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
cluster = distributed.LocalCluster()
client = distributed.Client(cluster)
client

# Load data

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/da_enkf_for_soil/'
util_dir = '/work/um0203/u300636/for2131/runs/utilities'

## Nature run

In [ ]:
vr_h2o_path = os.path.join(base_path, '016', 'h2o_cleaned.nc')
vr_h2o = xr.open_dataset(vr_h2o_path)['H2OSOI'].squeeze(drop=True).isel(levsoi=4).chunk((1, 302, 267))

## LETKF

In [ ]:
da_h2o_letkf_vr_path = os.path.join(base_path, '020', 'da_offline_3d_enkf_nature.nc')
da_h2o_letkf_vr = xr.open_dataset(da_h2o_letkf_vr_path)['H2OSOI'].squeeze(drop=True).chunk((1, 40, 302, 267))

In [ ]:
da_h2o_letkf_1d_path = os.path.join(base_path, '020', 'da_offline_1d_enkf_nature.nc')
da_h2o_letkf_1d = xr.open_dataarray(da_h2o_letkf_1d_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_letkf_1d_h2o_path = os.path.join(base_path, '020', 'da_offline_1d_enkf_h2o.nc')
da_h2o_letkf_1d_h2o = xr.open_dataarray(da_h2o_letkf_1d_h2o_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_letkf_ana_path = os.path.join(base_path, '020', 'da_offline_3d_enkf.nc')
da_h2o_letkf_ana = xr.open_dataarray(da_h2o_letkf_ana_path).squeeze(drop=True).chunk((40, 1, 302, 267))

# SEKF

In [ ]:
da_h2o_sekf_vr_path = os.path.join(base_path, '023', 'juwels', 'da_offline_sekf_nature.nc')
da_h2o_sekf_vr = xr.open_dataarray(da_h2o_sekf_vr_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_sekf_enkf_path = os.path.join(base_path, '023', 'juwels', 'da_offline_enkf_nature.nc')
da_h2o_sekf_enkf = xr.open_dataarray(da_h2o_sekf_enkf_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_sekf_3d_path = os.path.join(base_path, '023', 'juwels', 'da_offline_letkf_nature.nc')
da_h2o_sekf_3d = xr.open_dataarray(da_h2o_sekf_3d_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_sekf_ecmwf_path = os.path.join(base_path, '023', 'juwels', 'da_offline_senkf.nc')
da_h2o_sekf_ecmwf = xr.open_dataarray(da_h2o_sekf_ecmwf_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_sekf_ana_path = os.path.join(base_path, '023', 'juwels', 'da_offline_sekf.nc')
da_h2o_sekf_ana = xr.open_dataarray(da_h2o_sekf_ana_path).squeeze(drop=True).chunk((1, 302, 267))

In [ ]:
da_h2o_letkf_ana['time'] = da_h2o_letkf_1d.indexes['time']
da_h2o_sekf_ana['time'] = da_h2o_sekf_vr.indexes['time']

# Analyse error sources

In [ ]:
letkf_ana_err = da_h2o_letkf_ana.mean('ensemble') - vr_h2o
letkf_vr_err = da_h2o_letkf_vr.mean('ensemble') - vr_h2o
letkf_1d_err = da_h2o_letkf_1d - vr_h2o
letkf_h2o_err = da_h2o_letkf_1d_h2o - vr_h2o

In [ ]:
sekf_ana_err = da_h2o_sekf_ana - vr_h2o
sekf_vr_err = da_h2o_sekf_vr - vr_h2o
sekf_ens_err = da_h2o_sekf_enkf - vr_h2o
sekf_3d_err = da_h2o_sekf_3d - vr_h2o
sekf_ecmwf_err = da_h2o_sekf_ecmwf - vr_h2o

## Temporal evolution

In [ ]:
rmse_letkf_ana = np.sqrt((letkf_ana_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_letkf_vr = np.sqrt((letkf_vr_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_letkf_1d = np.sqrt((letkf_1d_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_letkf_h2o = np.sqrt((letkf_h2o_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_sekf_ana = np.sqrt((sekf_ana_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_sekf_vr = np.sqrt((sekf_vr_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_sekf_ens = np.sqrt((sekf_ens_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_sekf_3d = np.sqrt((sekf_3d_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()
rmse_sekf_ecmwf = np.sqrt((sekf_ecmwf_err[..., 30:-30, 30:-30]**2).mean(dim=['lat', 'lon'])).load()

In [ ]:
figsize = [s for s in plt.rcParams['figure.figsize']]
figsize[1] *= 0.48

fig, ax = plt.subplots(ncols=2, figsize=figsize)
ax[0].plot(rmse_sekf_3d.time, rmse_sekf_3d, label='SEKF-3D-EnKF', c='deepskyblue', ls='-.', lw=0.5)
ax[0].plot(rmse_sekf_ens.time, rmse_sekf_ens, label='SEKF-1D-EnKF', c='royalblue', ls='--', lw=0.5)
ax[0].plot(rmse_sekf_ecmwf.time, rmse_sekf_ecmwf, label='SEKF-ENS', c='firebrick', ls='-.', lw=0.5)
ax[0].plot(rmse_sekf_vr.time, rmse_sekf_vr, label='SEKF-nature', c='grey', ls='dotted', lw=0.5)
ax[0].plot(rmse_sekf_ana.time, rmse_sekf_ana, label='SEKF-Analysis', c='black', ls='-', lw=0.5)

ax[1].plot(rmse_letkf_h2o.time, rmse_letkf_h2o, label='LETKF-1D-H2O', c='salmon', ls='-.', lw=0.5)
ax[1].plot(rmse_letkf_1d.time, rmse_letkf_1d, label='LETKF-1D-nature', c='royalblue', ls='--', lw=0.5)
ax[1].plot(rmse_letkf_vr.time[1:], rmse_letkf_vr[1:], label='LETKF-3D-nature', c='deepskyblue', ls='-.', lw=0.5)
ax[1].plot(rmse_letkf_ana.time, rmse_letkf_ana, label='LETKF-Analysis', c='black', ls='-', lw=0.5)

ax[0].text(x=0.02, y=0.98, s='(a)', va='top', ha='left', transform=ax[0].transAxes)
ax[1].text(x=0.02, y=0.98, s='(b)', va='top', ha='left', transform=ax[1].transAxes)

ax[0].set_ylabel('RMSE (m$^3$/m$^3$)')
ax[0].set_ylim(0.00, 0.025)
ax[1].set_ylim(0.00, 0.025)
ax[0].set_yticks(np.arange(0.0, 0.025, 0.005))
ax[1].set_yticks([])

ax[0].set_xticks([])
ax[0].set_xlim(pd.to_datetime('2015-07-30 18:00'), pd.to_datetime('2015-08-07 06:00'))
ax[1].set_xlim(pd.to_datetime('2015-07-31 06:00'), pd.to_datetime('2015-08-08 00:00'))

#ax[1].set_xlabel('Time (2015-07-31 – 2015-08-07)')
ax[1].xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

#ax[0].set_xlabel('Time (2015-07-31 – 2015-08-07)')
ax[0].text(x=1, y=-0.18, s='Time (2015-07-31 – 2015-08-07)', va='center', ha='center', transform=ax[0].transAxes)
ax[0].xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

ax[0].legend(ncol=2)
ax[1].legend()

fig.subplots_adjust(wspace=0)
fig.savefig('../figures/fig_07_offline_exp.png', dpi=300)
plt.show()